In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.abspath('../src'))

from data_loader import DiabetesLoader, TestLoader

loader = DiabetesLoader('../data/raw/train.csv')
df = loader.get_df()

test_loader = TestLoader('../data/raw/test.csv')
test,id = test_loader.get_data() 

Loading data from /home/jupyter/work/data/raw/train.csv
Data Loaded. Shape: (700000, 25)
Loading data from /home/jupyter/work/data/raw/test.csv
Test X: (300000, 25)


In [2]:
print(f"train shape: {df.shape} (+1 from target)")
print(f"test shape: {test.shape}")

train shape: (700000, 25) (+1 from target)
test shape: (300000, 24)


In [3]:
from autogluon.tabular import TabularPredictor
from config import TARGET

predictor = TabularPredictor(label=TARGET, eval_metric='roc_auc')

No path specified. Models will be saved in: "AutogluonModels/ag-20251228_062721"


In [4]:
predictor.fit(
    train_data=df, 
    presets='best_quality',
    # excluded_model_types=['RF', 'XT', 'KNN', 'NN_TORCH'],
    time_limit=900  # 15 minutes
)

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 900 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20251228_062721/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 225s
AutoGluon will save models to "AutogluonModels/ag-20251228_062721/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.8
Operating System:   Linux
Platform Machine:   x86_6

In [5]:
import pandas as pd

predictions = predictor.predict_proba(test)


In [6]:
submission = pd.DataFrame({
    'id': id,
    TARGET: predictions[1] 
})

print(submission)

            id  diagnosed_diabetes
0       700000            0.541286
1       700001            0.596761
2       700002            0.680291
3       700003            0.515751
4       700004            0.884711
...        ...                 ...
299995  999995            0.728116
299996  999996            0.637711
299997  999997            0.584599
299998  999998            0.621204
299999  999999            0.556383

[300000 rows x 2 columns]


In [7]:
submission.to_csv('../outputs/autogluon_full_roc_auc.csv', index=False)